In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Kuramoto-Sivashinsky Denklemi PINN Modeli
# ∂u/∂t + u*∂u/∂x + ∂²u/∂x² + ∂⁴u/∂x⁴ = 0
# Kaotik davranış gösteren 4. dereceden nonlinear PDE

print("Kuramoto-Sivashinsky Denklemi PINN Modeli")
print("=" * 50)
print("PDE: ∂u/∂t + u*∂u/∂x + ∂²u/∂x² + ∂⁴u/∂x⁴ = 0")
print("Domain: x ∈ [0, 2π], t ∈ [0, T]")
print("=" * 50)


# Model parametreleri
L = 2 * np.pi  # Spatial domain [0, 2π]
T = 10.0       # Time domain [0, T]
n_hidden = 128 # Hidden layer boyutu
n_layers = 4   # Layer sayısı

# Neural Network modeli
def create_ks_model():
    """Kuramoto-Sivashinsky için özel PINN modeli"""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(n_hidden, activation='tanh', input_shape=(2,)),
        tf.keras.layers.Dense(n_hidden, activation='tanh'),
        tf.keras.layers.Dense(n_hidden, activation='tanh'),
        tf.keras.layers.Dense(n_hidden, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    return model

# Kuramoto-Sivashinsky PDE residual hesaplama
@tf.function
def ks_physics_loss(model, x, t):
    """
    KS denklemi: ∂u/∂t + u*∂u/∂x + ∂²u/∂x² + ∂⁴u/∂x⁴ = 0
    """
    with tf.GradientTape(persistent=True) as tape4:
        tape4.watch([x, t])
        with tf.GradientTape(persistent=True) as tape3:
            tape3.watch([x, t])
            with tf.GradientTape(persistent=True) as tape2:
                tape2.watch([x, t])
                with tf.GradientTape(persistent=True) as tape1:
                    tape1.watch([x, t])
                    u = model(tf.stack([x, t], axis=1))

                # 1. dereceden türevler
                du_dx = tape1.gradient(u, x)
                du_dt = tape1.gradient(u, t)

            # 2. dereceden türevler
            d2u_dx2 = tape2.gradient(du_dx, x)

        # 3. dereceden türev
        d3u_dx3 = tape3.gradient(d2u_dx2, x)

    # 4. dereceden türev
    d4u_dx4 = tape4.gradient(d3u_dx3, x)

    # Kuramoto-Sivashinsky denklemi
    # ∂u/∂t + u*∂u/∂x + ∂²u/∂x² + ∂⁴u/∂x⁴ = 0
    pde_residual = du_dt + u * du_dx + d2u_dx2 + d4u_dx4

    # Cleanup
    del tape1, tape2, tape3, tape4

    return tf.reduce_mean(tf.square(pde_residual))

# Periyodik sınır koşulu
def periodic_boundary_loss(model, x_left, x_right, t):
    """Periyodik sınır koşulu: u(0,t) = u(2π,t)"""
    u_left = model(tf.stack([x_left, t], axis=1))
    u_right = model(tf.stack([x_right, t], axis=1))

    return tf.reduce_mean(tf.square(u_left - u_right))

# Periyodik sınır koşulu türevler için
@tf.function
def periodic_derivative_loss(model, x_left, x_right, t):
    """Periyodik sınır koşulu türevler için: ∂u/∂x(0,t) = ∂u/∂x(2π,t)"""
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x_left, x_right])
        u_left = model(tf.stack([x_left, t], axis=1))
        u_right = model(tf.stack([x_right, t], axis=1))

    du_dx_left = tape.gradient(u_left, x_left)
    du_dx_right = tape.gradient(u_right, x_right)

    del tape

    return tf.reduce_mean(tf.square(du_dx_left - du_dx_right))

# Başlangıç koşulu
def initial_condition_loss(model, x, t):
    """Başlangıç koşulu: u(x,0) = cos(x/16)*(1+sin(x/16))"""
    u_pred = model(tf.stack([x, t], axis=1))

    # Başlangıç koşulu - kaotik davranış için pertürbasyonlu
    u_initial = tf.cos(x/16) * (1 + tf.sin(x/16))

    return tf.reduce_mean(tf.square(u_pred - u_initial))

# Toplam loss fonksiyonu
def total_loss(model, x_pde, t_pde, x_ic, t_ic, x_left, x_right, t_bc):
    """Toplam loss: Physics + IC + BC"""

    # Physics loss
    physics = ks_physics_loss(model, x_pde, t_pde)

    # Initial condition loss
    initial = initial_condition_loss(model, x_ic, t_ic)

    # Periodic boundary losses
    boundary_val = periodic_boundary_loss(model, x_left, x_right, t_bc)
    boundary_der = periodic_derivative_loss(model, x_left, x_right, t_bc)

    return physics + 100 * initial + 50 * (boundary_val + boundary_der)

# Eğitim verisi oluşturma
def generate_ks_training_data():
    """KS denklemi için eğitim verisi"""

    # PDE noktaları (domain içi)
    n_pde = 5000
    x_pde = tf.random.uniform([n_pde], 0, L, dtype=tf.float32)
    t_pde = tf.random.uniform([n_pde], 0, T, dtype=tf.float32)

    # Başlangıç koşulu noktaları (t=0)
    n_ic = 500
    x_ic = tf.random.uniform([n_ic], 0, L, dtype=tf.float32)
    t_ic = tf.zeros([n_ic], dtype=tf.float32)

    # Periyodik sınır koşulu noktaları
    n_bc = 200
    x_left = tf.zeros([n_bc], dtype=tf.float32)
    x_right = tf.ones([n_bc], dtype=tf.float32) * L
    t_bc = tf.random.uniform([n_bc], 0, T, dtype=tf.float32)

    return x_pde, t_pde, x_ic, t_ic, x_left, x_right, t_bc

# Eğitim adımı
@tf.function
def train_step(model, optimizer, x_pde, t_pde, x_ic, t_ic, x_left, x_right, t_bc):
    with tf.GradientTape() as tape:
        loss = total_loss(model, x_pde, t_pde, x_ic, t_ic, x_left, x_right, t_bc)

    gradients = tape.gradient(loss, model.trainable_variables)

    # Gradient clipping (numerical stability için)
    gradients = [tf.clip_by_norm(g, 1.0) for g in gradients]

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

# Model ve optimizer oluşturma
print("\nModel oluşturuluyor...")
model = create_ks_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Eğitim verisi
print("Eğitim verisi hazırlanıyor...")
x_pde, t_pde, x_ic, t_ic, x_left, x_right, t_bc = generate_ks_training_data()

# Eğitim döngüsü
print("KS-PINN eğitimi başlıyor...")
print("Bu kaotik bir sistem olduğu için eğitim biraz uzun sürebilir...")

losses = []
epochs = 2000

for epoch in range(epochs):
    loss = train_step(model, optimizer, x_pde, t_pde, x_ic, t_ic, x_left, x_right, t_bc)
    losses.append(loss.numpy())

    # Learning rate scheduling
    if epoch % 500 == 0 and epoch > 0:
        optimizer.learning_rate = optimizer.learning_rate * 0.8

    # Veri yenileme (kaotik sistem için önemli)
    if epoch % 1000 == 0 and epoch > 0:
        x_pde, t_pde, x_ic, t_ic, x_left, x_right, t_bc = generate_ks_training_data()

    if epoch % 100 == 0:
        print(f"Epoch {epoch:5d}, Loss: {loss:.6f}, LR: {optimizer.learning_rate.numpy():.6f}")

print("Eğitim tamamlandı!")

# Sonuçları değerlendirme ve görselleştirme
def evaluate_ks_solution():
    """KS çözümünü değerlendir ve görselleştir"""

    # Test grid'i
    nx, nt = 200, 100
    x_test = np.linspace(0, L, nx)
    t_test = np.linspace(0, T, nt)
    X, T_grid = np.meshgrid(x_test, t_test)

    # PINN tahminleri
    print("PINN çözümü hesaplanıyor...")
    test_points = np.column_stack([X.flatten(), T_grid.flatten()])
    u_pred = model(test_points.astype(np.float32)).numpy().reshape(X.shape)

    return x_test, t_test, X, T_grid, u_pred

# Görselleştirme fonksiyonları
def plot_ks_results(x_test, t_test, X, T_grid, u_pred):
    """KS sonuçlarını görselleştir"""

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Spatio-temporal evolution
    im1 = axes[0,0].contourf(X, T_grid, u_pred, levels=50, cmap='RdBu_r')
    axes[0,0].set_title('KS Denklemi - Spatio-Temporal Evolution')
    axes[0,0].set_xlabel('x')
    axes[0,0].set_ylabel('t')
    plt.colorbar(im1, ax=axes[0,0])

    # 2. Solution at different times
    times_to_plot = [0, T/4, T/2, 3*T/4, T]
    colors = ['blue', 'green', 'red', 'purple', 'orange']

    for i, t_plot in enumerate(times_to_plot):
        t_idx = int(t_plot * len(t_test) / T)
        if t_idx >= len(t_test):
            t_idx = len(t_test) - 1

        axes[0,1].plot(x_test, u_pred[t_idx, :],
                      color=colors[i], linewidth=2,
                      label=f't = {t_plot:.1f}')

    axes[0,1].set_title('Çözüm Profilleri (Farklı Zamanlarda)')
    axes[0,1].set_xlabel('x')
    axes[0,1].set_ylabel('u(x,t)')
    axes[0,1].legend()
    axes[0,1].grid(True)

    # 3. Time evolution at fixed points
    x_points = [L/4, L/2, 3*L/4]
    for i, x_point in enumerate(x_points):
        x_idx = int(x_point * len(x_test) / L)
        axes[1,0].plot(t_test, u_pred[:, x_idx],
                      linewidth=2, label=f'x = {x_point:.2f}')

    axes[1,0].set_title('Zaman Evrim (Sabit Noktalarda)')
    axes[1,0].set_xlabel('t')
    axes[1,0].set_ylabel('u(x,t)')
    axes[1,0].legend()
    axes[1,0].grid(True)

    # 4. Loss curve
    axes[1,1].plot(losses, 'b-', linewidth=2)
    axes[1,1].set_title('Eğitim Loss Eğrisi')
    axes[1,1].set_xlabel('Epoch')
    axes[1,1].set_ylabel('Loss')
    axes[1,1].set_yscale('log')
    axes[1,1].grid(True)

    plt.tight_layout()
    plt.show()

# 3D görselleştirme
def plot_3d_ks(x_test, t_test, X, T_grid, u_pred):
    """3D görselleştirme"""
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Surface plot
    surf = ax.plot_surface(X, T_grid, u_pred, cmap='RdBu_r',
                          alpha=0.8, edgecolor='none')

    ax.set_title('Kuramoto-Sivashinsky Denklemi - 3D Görünüm')
    ax.set_xlabel('x (Spatial)')
    ax.set_ylabel('t (Time)')
    ax.set_zlabel('u(x,t)')

    # Colorbar
    fig.colorbar(surf, ax=ax, shrink=0.5)

    plt.show()

# Kaotik özellikler analizi
def analyze_chaos(x_test, t_test, u_pred):
    """Kaotik özellikleri analiz et"""
    print("\n" + "="*50)
    print("KAOTİK ÖZELLİKLER ANALİZİ")
    print("="*50)

    # Energy hesapla
    energy = np.trapz(u_pred**2, x_test, axis=1)

    # Dissipation rate hesapla (yaklaşık)
    dissipation = -np.gradient(energy, t_test)

    print(f"Maksimum amplitude: {np.max(np.abs(u_pred)):.4f}")
    print(f"Minimum amplitude: {np.min(u_pred):.4f}")
    print(f"Başlangıç energy: {energy[0]:.4f}")
    print(f"Son energy: {energy[-1]:.4f}")
    print(f"Ortalama dissipation: {np.mean(dissipation[dissipation>0]):.4f}")

    # Energy evolution plot
    plt.figure(figsize=(10, 6))

    plt.subplot(2, 1, 1)
    plt.plot(t_test, energy, 'b-', linewidth=2)
    plt.title('Energy Evolution')
    plt.xlabel('t')
    plt.ylabel('Energy')
    plt.grid(True)

    plt.subplot(2, 1, 2)
    plt.plot(t_test[1:], dissipation[1:], 'r-', linewidth=2)
    plt.title('Dissipation Rate')
    plt.xlabel('t')
    plt.ylabel('dE/dt')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Ana çalıştırma
if __name__ == "__main__":
    # Çözümü hesapla
    x_test, t_test, X, T_grid, u_pred = evaluate_ks_solution()

    # Görselleştirmeler
    plot_ks_results(x_test, t_test, X, T_grid, u_pred)
    plot_3d_ks(x_test, t_test, X, T_grid, u_pred)

    # Kaotik analiz
    analyze_chaos(x_test, t_test, u_pred)

    print("\n🎉 Kuramoto-Sivashinsky PINN modeli başarıyla tamamlandı!")
    print("Bu kaotik bir sistem olduğu için:")
    print("- Karmaşık spatio-temporal yapılar görmelisiniz")
    print("- Düzensiz oscillasyonlar")
    print("- Energy dissipation")

Kuramoto-Sivashinsky Denklemi PINN Modeli
PDE: ∂u/∂t + u*∂u/∂x + ∂²u/∂x² + ∂⁴u/∂x⁴ = 0
Domain: x ∈ [0, 2π], t ∈ [0, T]

Model oluşturuluyor...
Eğitim verisi hazırlanıyor...
KS-PINN eğitimi başlıyor...
Bu kaotik bir sistem olduğu için eğitim biraz uzun sürebilir...
Epoch     0, Loss: 107.529999, LR: 0.001000


KeyboardInterrupt: 